![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and `ibm/granite-3-3-8b-instruct` to perform chat conversation with control roles

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook provides a detailed demonstration of the steps and code required to showcase support for control roles available in IBM Granite models.

Some familiarity with Python is helpful. This notebook uses Python 3.12.

## Learning goal

The purpose of this notebook is to demonstrate how to use control roles available in IBM Granite models.

## Table of Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models on watsonx](#models)
- [Work with chat messages](#chat)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://cloud.ibm.com/catalog/services/watsonxai-runtime" target="_blank" rel="noopener no referrer">watsonx.ai Runtime Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/wml-plans.html?context=wx&audience=wdp" target="_blank" rel="noopener no referrer">here</a>).

### Install dependencies

In [1]:
%pip install -U ibm-watsonx-ai | tail -n 1

### Define the watsonx.ai credentials
Use the code cell below to define the watsonx.ai credentials that are required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">Managing user API keys</a>.

In [2]:
import getpass
from ibm_watsonx_ai import Credentials

credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key=getpass.getpass("Enter your watsonx.ai api key and hit enter: "),
)

### Define the project ID
You need to provide the project ID to give the Foundation Model the context for the call. If you have a default project ID set in Watson Studio, the notebook obtains that project ID. Otherwise, you need to provide the project ID in the code cell below.

In [3]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Enter your project_id and hit enter: ")

<a id="models"></a>
## Set up the Foundation Model on `watsonx.ai`


Specify the `model_id` of the model you will use for the chat with tools.

In [4]:
model_id = "ibm/granite-3-3-8b-instruct"

### Define the model parameters

You might need to adjust model parameters depending on the model you use.

In [5]:
from ibm_watsonx_ai.foundation_models.schema import TextChatParameters

TextChatParameters.show()

+-------------------+----------------------------------------+------------------------------+
| PARAMETER         | TYPE                                   | EXAMPLE VALUE                |
+===================+========================================+==============================+
| frequency_penalty | float, NoneType                        | 0.5                          |
+-------------------+----------------------------------------+------------------------------+
| logprobs          | bool, NoneType                         | True                         |
+-------------------+----------------------------------------+------------------------------+
| top_logprobs      | int, NoneType                          | 3                            |
+-------------------+----------------------------------------+------------------------------+
| presence_penalty  | float, NoneType                        | 0.3                          |
+-------------------+---------------------------------------

In [6]:
params = TextChatParameters(
    temperature=1
)

### Initialize the model

Initialize the `ModelInference` class with the previously set parameters.

In [7]:
from ibm_watsonx_ai.foundation_models import ModelInference

model = ModelInference(
    model_id=model_id,
    credentials=credentials,
    project_id=project_id,
    params=params
)

<a id="chat"></a>
## Work with chat messages 

### Work with a simple chat message using `chat_stream`

In [8]:
messages = [
    {
        "role": "user", 
        "content": "Which Formula 1 team is partnered with IBM?"
    }
]

chat_stream_response = model.chat_stream(messages=messages)

In [9]:
for chunk in chat_stream_response:
    if chunk["choices"]:
        print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

Scuderia Ferrari, an Italian Formula One racing team, collaborates with IBM for technology and data analytics in Formula 1.

### Work with chat message with `thinking` control role
The `thinking` control role enables enhanced reasoning in model responses.

In [10]:
messages = [
    {
        "role": "control",
        "content": "thinking",
    },
    {
        "role": "user", 
        "content": "Tell me a joke"
    }
]

chat_stream_response = model.chat_stream(messages=messages)

In [11]:
for chunk in chat_stream_response:
    if chunk["choices"]:
        print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

<think>Crafting a joke that aligns with the user's request involves considering elements such as surprise, humor, and appropriateness. Here's a classic light-hearted math joke:

Why was the equal sign so humble?

Because it knew it wasn't less than or greater than anyone else!

This joke relies on a play on words with mathematical symbols and portrays a personification, creating a humorous anthropomorphism of an equal sign.</think><response>### Joke:

Why was the equal sign so humble?

Because it knew it wasn't less than or greater than anyone else!

### Explanation:

The humor in this joke comes from the personification of the equal sign (=), attributing human traits (humility) to a mathematical symbol. By stating that the equal sign is "humble" because it understands it's neither less than (<) nor greater than (>) anything else, the joke cleverly utilizes mathematical operators, playing with their usual roles in equations.</response>

### Work with chat message with `length` control role
Using the `length` control role, you can specify how long the model response should be.

#### Short length

In [12]:
messages = [
    {
        "role": "control",
        "content": "length short",
    },
    {
        "role": "user", 
        "content": "Explain torque"
    }
]

chat_stream_response = model.chat_stream(messages=messages)

In [13]:
for chunk in chat_stream_response:
    if chunk["choices"]:
        print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

Torque is a measure of the force that can cause an object to rotate about an axis. It's calculated as the product of the force and the distance from the axis (torque = force × distance). The unit of torque is the Newton-meter (Nm).

#### Long length

In [14]:
messages = [
    {
        "role": "control",
        "content": "length long",
    },
    {
        "role": "user", 
        "content": "Explain torque"
    }
]

chat_stream_response = model.chat_stream(messages=messages)

In [15]:
for chunk in chat_stream_response:
    if chunk["choices"]:
        print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

Torque, also known as the moment of force, is a fundamental concept in physics and engineering, particularly in the study of rotational motion. It describes the rotational effect of a force applied to an object at a distance from its axis of rotation.

In simple terms, torque (τ) is a measure of how much a force acting on an object causes that object to rotate. It depends on two main factors: the magnitude of the force (F) and the distance (r) from the axis of rotation to the point where the force is applied, often called the lever arm. Mathematically, torque is given by the cross product of the force and the position vector (distance) from the axis: 

τ = r × F 

The unit of torque in the International System of Units (SI) is the Newton-meter (Nm).

Here are some key points to understand torque better:

1. **Direction:** Torque is a vector quantity, meaning it has both a magnitude and a direction. The direction of torque is perpendicular to the plane containing the force and the dista

### Work with chat message with `originality` control role
The `originality` control role specifies the type of summarization of the model response.

#### Extractive originality
When using `extractive` option, the model will create a summary of a given topic using the exact words used in its input data and may also cite its references.

In [16]:
messages = [
    {
        "role": "control",
        "content": "originality extractive",
    },
    {
        "role": "user", 
        "content": "Mineral composition of granites"
    }
]

chat_stream_response = model.chat_stream(messages=messages)

In [17]:
for chunk in chat_stream_response:
    if chunk["choices"]:
        print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

Granites are felsic igneous rocks, which means they are primarily composed of silicate minerals containing a significant amount of quartz, feldspar, and mica. Here's a typical mineral composition of granites:

1. **Quartz (SiO2)**: 20-40% - If present in more than 30%, the rock is considered quartzofeldspathic granite.

2. **Feldspars (K-feldspar (KAlSi3O8) and plagioclase feldspar (NaAlSi3O8 - CaAl2Si2O8)):** 
   - K-feldspar: 10-25%
   - Plagioclase feldspar: 10-20%
   - The feldspars provide the primary source of potassium, sodium, and aluminium in granites.

3. **Micas (Biotite and muscovite)**: 5-15% - Biotite is usually dark, while muscovite is lighter-colored. Micas contribute to granite's platy texture and add iron, magnesium, and aluminum.

4. **Amphiboles (Hornblende)**: 0-10% - Some granites contain hornblende amphiboles, which can be greenish or dark-colored. This mineral adds iron, magnesium, and sodium.

5. **Other minerals**: Small amounts of garnet, zircon, sphene, and 

#### Abstractive originality
When using `abstractive` originality, the model will paraphrase its data in order to create its response.

In [18]:
messages = [
    {
        "role": "control",
        "content": "originality abstractive",
    },
    {
        "role": "user", 
        "content": "Mineral composition of granites"
    }
]

chat_stream_response = model.chat_stream(messages=messages)

In [19]:
for chunk in chat_stream_response:
    if chunk["choices"]:
        print(chunk["choices"][0]["delta"].get("content", ""), end="", flush=True)

Granite is a common type of intrusive igneous rock, characterized by its coarse-grained texture, and a mineral composition predominantly consisting of quartz, feldspar, and mica. Here's a more detailed breakdown of the typical mineral components:

1. Quartz: Quartz is a crucial component in granite, typically making up 20-30% of its composition. It's a hard, clear, or translucent mineral, which provides resistance to weathering.

2. Feldspar: Feldspars are the second most abundant minerals in granite, making up around 40-60%. They include two main types of feldspar: plagioclase feldspar (calcium-rich and sodium-rich varieties) and potassium feldspar (orthoclase or microcline). Feldspars lend granite its distinct color, which can vary from pink and light red to white or gray, depending on the types and amounts of feldspars present.

3. Mica: Micas are a group of minerals that include biotite (black or dark brown) and muscovite (light-colored or clear, often referred to as "book mica" du

<a id="summary"></a>
## Summary and next steps

You successfully completed this notebook!

You learned how to work with chat models using tools and watsonx.ai.

Check out our _<a href="https://ibm.github.io/watsonx-ai-python-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Author

**Rafał Chrzanowski**, Software Engineer Intern at watsonx.ai.

Copyright © 2025 IBM. This notebook and its source code are released under the terms of the MIT License.